# Notebook details

In [ ]:
def setup_notebook(fix_python_path=True, reduce_margins=True, plot_inline=True):
    if reduce_margins:
        # Reduce side margins of the notebook
        from IPython.core.display import display, HTML
        display(HTML("<style>.container { width:100% !important; }</style>"))

    if fix_python_path:
        # add egosocial to the python path
        import os, sys
        sys.path.extend([os.path.dirname(os.path.abspath('.'))])

    if plot_inline:
        # Plots inside cells
        %matplotlib inline

setup_notebook()

# Imports and Constants Definition

In [ ]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

import argparse
from collections import defaultdict
from functools import partial
import logging
import matplotlib.pyplot as plt
import os
import shutil
import sys
import threading

import numpy as np
from IPython.display import SVG

import keras
from keras import backend as K
from keras.applications import imagenet_utils
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Cropping2D 
from keras.layers import GlobalMaxPooling2D
from keras.layers.noise import AlphaDropout
from keras.models import Model
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot

import egosocial
import egosocial.config
from egosocial.core.types import relation_to_domain
from egosocial.core.types import relation_to_domain_vec
from egosocial.utils.filesystem import create_directory 
from egosocial.utils.filesystem import check_directory
from egosocial.utils.logging import setup_logging
from egosocial.utils.keras.autolosses import AutoMultiLossWrapper
from egosocial.utils.keras.backend import limit_gpu_allocation_tensorflow
from egosocial.utils.keras.callbacks import PlotLearning
from egosocial.utils.keras.layers import LRN
from egosocial.utils.keras.utils import flow_from_dirs
from egosocial.utils.keras.utils import fuse_inputs_generator

# TODO: move these constants to the library
DOMAIN, RELATION = 'domain', 'relation'
END_TO_END, ATTRIBUTES = 'end_to_end', 'attributes'
N_CLS_RELATION, N_CLS_DOMAIN = 16, 5
RELATION_LABELS = [str(label) for label in range(N_CLS_RELATION)]
FILE_FORMAT = '{prefix}{dtype}{idx}_{split}_{n_cls}{ext}'

MODEL_INPUTS = ('body/1', 'body/2', 'face/1', 'face/2')
IMAGE_SHAPE = (256, 256, 3)

# it should be (227, 227, 3), but the model adds padding anyway
INPUT_SHAPE = (228, 228, 3)
SHARED_SEED = 1
STREAM_IDS = [str(idx) for idx in [1, 2]]

DEPRECATED_HOME = '/root/'
NEW_HOME='/home/'

# Limit GPU memory allocation with Tensorflow

In [ ]:
if K.backend() == 'tensorflow':
    limit_gpu_allocation_tensorflow(0.5)

# Input arguments and fake main

In [ ]:
def get_file(split, idx=1, dtype='body', n_cls=N_CLS_RELATION, 
             ext='.txt', prefix='single_'):
    return FILE_FORMAT.format(split=split, prefix=prefix, dtype=dtype, 
                              idx=idx, n_cls=n_cls, ext=ext)

class Configuration:
    def __init__(self, args):
        # setup directories
        self.PROJECT_DIR = args.project_dir

        self.SPLITS_DIR = os.path.join(self.PROJECT_DIR,
                                       'datasets/splits/annotator_consistency3')

        # splits (switch from caffe's split name convention to keras's convention)        
        _train, _val, _test = 'train', 'test', 'eval'        
        self.LABEL_FILES = {split: os.path.join(self.SPLITS_DIR, get_file(split=split))
                            for split in (_train, _val, _test)}

        self.EPOCHS = args.epochs
        self.BATCH_SIZE = args.batch_size
        self.LR = args.lr
        
        # reuse precomputed model?
        self.REUSE_MODEL = args.reuse_model
        # save model to disk?
        self.SAVE_MODEL = args.save_model
        # save model statistics to disk?
        self.SAVE_STATS = args.save_stats
        
        self.FAKE_DIR = os.path.join(self.PROJECT_DIR, 'datasets', 'fake_dir')
        
        self.MODELS_CACHE_DIR = os.path.join(self.PROJECT_DIR, 
                                             'models/keras_models/models_keras_repr')
        self.MODELS_CACHE_SUBDIR = 'bins' 

def non_negative_float(value):
    fvalue = float(value)
    if fvalue < 0:
        raise argparse.ArgumentTypeError(
            "%s is an invalid non negative float value" % value)
    return fvalue
        
def positive_int(value):
    ivalue = int(value)
    if ivalue <= 0:
        raise argparse.ArgumentTypeError(
            "%s is an invalid positive int value" % value)
    return ivalue

def main(*fake_args):
    setup_logging(egosocial.config.LOGGING_CONFIG)

    entry_msg = 'Reproduce experiments in Social Relation Recognition paper.'
    parser = argparse.ArgumentParser(description=entry_msg)

    parser.add_argument('--project_dir', required=True,
                        help='Base directory.')

    parser.add_argument('--reuse_model', required=False,
                        action='store_true',
                        help='Use precomputed model if available.')

    parser.add_argument('--save_model', required=False,
                        action='store_true',
                        help='Save model to disk.')

    parser.add_argument('--save_stats', required=False,
                        action='store_true',
                        help='Save statistics to disk.')

    parser.add_argument('--epochs', required=False, type=positive_int,
                        default=100,
                        help='Max number of epochs.')

    parser.add_argument('--batch_size', required=False, type=positive_int,
                        default=32,
                        help='Batch size.')
    
    parser.add_argument('--lr', required=False, type=non_negative_float,
                        default=0.001,
                        help='Initial learning rate.')    
    
    # TODO: implement correctly
    args = parser.parse_args(*fake_args)
    # keep configuration
    conf = Configuration(args)
    # check directories
    check_directory(conf.PROJECT_DIR, 'Project')
    check_directory(conf.SPLITS_DIR, 'Splits')
    check_directory(os.path.join(conf.MODELS_CACHE_DIR, conf.MODELS_CACHE_SUBDIR), 'Snapshots')
    
    return conf

# Helper functions

In [ ]:
# TODO: move to utils
class AttributeSelector:
    def __init__(self, all_attrs):
        body_attributes = self.filter_by_keyword(all_attrs, 'body')
        face_attributes = self.filter_by_keyword(all_attrs, 'face')
        face_attributes.extend(self.filter_by_keyword(all_attrs, 'head'))

        self._selector = {'all': all_attrs,
                          'body': body_attributes,
                          'face': face_attributes}

    def filter(self, query):
        query = query.lower()
        if query in self._selector:
            selected_attributes = self._selector[query]
        else:
            selected_attributes = self.filter_by_keyword(self._selector['all'],
                                                         query)

        return selected_attributes

    def filter_by_keyword(self, attribute_list, key):
        return [attr_name for attr_name in attribute_list if key in attr_name]

# TODO: move to utils
def domain_to_relation_map():
    W = [np.zeros(N_CLS_RELATION) for _ in range(N_CLS_DOMAIN)]
    for rel in range(N_CLS_RELATION):
        dom = relation_to_domain(rel)
        W[dom] += to_categorical(rel, N_CLS_RELATION)
    return np.array(W).T    

# TODO: move to utils
def get_relation_domain(data_batch, inputs, mode=None):
    mode = mode if mode else 'both_splitted'
    assert mode in ('both_splitted', 'both_fused', 'relation', 'domain')
    # relation output (every attribute has the same output)
    y_rel = data_batch[0][1] # indices 0: first input, 1: label data
    
    if mode in ('both_splitted', 'both_fused', 'domain'):
        # inverse of function to_categorical
        plain_y_rel = np.argmax(y_rel, axis=1)
        # final output is domain, relation
        y_dom = to_categorical(relation_to_domain_vec(plain_y_rel), N_CLS_DOMAIN)

    if mode == 'both_splitted':
        return dict(domain=y_dom, relation=y_rel)
    elif mode == 'both_fused':
        return dict(domain_relation=np.concatenate((y_dom, y_rel), axis=1))
    elif mode == 'domain':
        return dict(domain=y_dom)
    else:
        return dict(relation=y_rel)    


def create_data_split_generators(directory, 
                                 train_gen_args=None, 
                                 test_val_gen_args=None,
                                 fix_path_cbk=None,
                                 output_mode=None,
                                 **kwargs):
    check_directory(directory)
    
    inputs = list(MODEL_INPUTS)
    def input_dirs(split):
        return [os.path.join(directory, input_name, split) 
                for input_name in inputs]

    # splits (switch from caffe's split name convention to keras's convention)    
    _train, _val, _test = 'train', 'test', 'eval'
        
    split_gen_args = [ (_train, train_gen_args)
                     , (_val, test_val_gen_args)
                     , (_test, test_val_gen_args)]

    split_generators = []
    for split, gen_args in split_gen_args:
        gens = flow_from_dirs(
            input_dirs(split), 
            gen_args=gen_args,
            fix_path_cbk=fix_path_cbk,
            **kwargs
        )
        
        fused_gen = fuse_inputs_generator(
            gens, inputs, 
            outputs_callback=partial(get_relation_domain, mode=output_mode)
        )
        
        split_generators.append(fused_gen)
        
    return split_generators

def process_line(line):
    image_path, label = line.strip().rsplit(' ', 1)

    if image_path.startswith(DEPRECATED_HOME):
        image_path = os.path.join(NEW_HOME, image_path[len(DEPRECATED_HOME):])

    return image_path, label

class FakeDirectory:

    def __init__(self, fake_dir, splits_dir):
        self.fake_dir = fake_dir
        self.splits_dir = splits_dir 
                
        self._fake_link_fmt = 'fakelink{}_{}'
        self._file_map = {}
        
    def create_structure(self):
        
        self._file_map = {}
        
        if os.path.isdir(self.fake_dir):
            shutil.rmtree(self.fake_dir)

        create_directory(self.fake_dir, 'Fake')

        for dtype in ('body', 'face'):
            dtype_dir = os.path.join(self.fake_dir, dtype)
            create_directory(dtype_dir, 'Fake {}'.format(dtype))

            # set dtype to body or face
            file_callback = lambda **kwargs: get_file(dtype=dtype, **kwargs)      

            mapping = self._get_split_mapping(file_callback=file_callback)

            for images_dir, images_file in mapping.items():
                fake_images_dir = os.path.join(dtype_dir, images_dir)

                images_file_path = os.path.join(self.splits_dir, images_dir, images_file)
                with open(images_file_path) as f:
                    for fake_id, line in enumerate(f):
                        image_path, label = process_line(line)
                        
                        fake_label_dir = os.path.join(fake_images_dir, str(label))

                        # keras asks for a different directory for each class
                        if not os.path.exists(fake_label_dir):
                            create_directory(fake_label_dir, 'Label')                            
                        
                        # create symlinks for every entry
                        # a name may appear in several entries, so an unique fake name is created
                        fake_name = self._fake_link_fmt.format(fake_id, os.path.basename(image_path))
                        fake_path = os.path.join(fake_label_dir, fake_name)
                        
                        # keep mapping fakefile -> image_path
                        self._file_map[fake_path] = image_path

                        os.symlink(image_path, fake_path)


    def _get_split_mapping(self, file_callback=get_file):
        # splits (switch from caffe's split name convention to keras's convention)
        _train, _val, _test = 'train', 'test', 'eval'

        check_directory(self.splits_dir, 'Splits')

        mapping = {}

        for split in (_train, _val, _test):
            for idx in STREAM_IDS:
                key = '{idx}{sep}{split}'.format(split=split, sep=os.path.sep, idx=idx)
                value =  os.path.join(self.splits_dir, file_callback(split=split, idx=idx))
                mapping[key] = value

        return mapping

    def fix_fake_link(self, directory, filename):

        fake_path = os.path.join(directory, filename)
        assert fake_path in self._file_map        
        image_path = self._file_map[fake_path]

        fake_label_dir = os.path.split(fake_path)[0]
        label = os.path.split(fake_label_dir)[1]

        original_dir, basename = os.path.split(image_path)

        # common ancestor
        commonprefix = os.path.commonprefix([original_dir, directory])
        # remove commonprefix
        fake_subdir = directory[len(commonprefix):]        
        original_subdir = original_dir[len(commonprefix):]
        # go up in fake path up to common prefix, then go down the real path 
        go_up = os.path.join(*['..' for folder in fake_subdir.split(os.path.sep)])    
        go_down = original_subdir

        fixed_filename = os.path.join(go_up, go_down, basename)
        return fixed_filename
    
def get_models(attributes, cache_dir, cache_subdir):
    models = {}

    for attr in attributes:
        file, url =  egosocial.config.MODELS[attr]        

        model_file = keras.utils.get_file(
            file, url,
            cache_dir=cache_dir, 
            cache_subdir=cache_subdir,
        )
        
        models[attr] = keras.models.load_model(
            model_file, 
            custom_objects={'LRN': LRN}
        )
    
    return models

# Main class

In [ ]:
class SocialClassifier:
    
    def __init__(self, data_dir):
        self._data_dir = data_dir

        self._log = logging.getLogger(self.__class__.__name__)
        
        # initialize when model is configured
        self._model_wrapper = None
        self.model = None
        
        self._train_gen, self._val_gen, self._test_gen = None, None, None
    
    def setup_datagen(self, **kwargs):
        self._log.debug("Creating data generators.")        

        preprocessing_function = kwargs.pop('preprocessing_function', None)
        
        train_gen_args = dict(
            rescale=1./255,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            horizontal_flip=True,
            preprocessing_function=preprocessing_function,
        )
        test_val_gen_args = dict(
            rescale=1./255,
            preprocessing_function=preprocessing_function,
        )
        
        split_gens = create_data_split_generators(
            self._data_dir, train_gen_args, test_val_gen_args,
            **kwargs
        )
        
        self._train_gen, self._val_gen, self._test_gen = split_gens    
        
    def set_model(self, model, optimizer='adam', 
                  loss='categorical_crossentropy', 
                  loss_weights='auto',
                  **kwargs):
        assert model
        self._log.debug("Initializing model.")
        
        # wrapper allows to train the loss weights
        self._model_wrapper = AutoMultiLossWrapper(model)
        self._model_wrapper.compile(optimizer, loss, 
                                    loss_weights=loss_weights, 
                                    **kwargs)

        self.model = self._model_wrapper.model
        self._log.info(self.model.summary())
        
    def fit(self, steps_per_epoch, validation_steps, **kwargs):
        assert self.model
        assert self._train_gen
        assert self._val_gen
        self._log.debug("Training model from scratch.")        

        # train the model on the new data for a few epochs
        return self.model.fit_generator(
            self._train_gen,
            steps_per_epoch=steps_per_epoch,
            validation_data=self._val_gen,
            validation_steps=validation_steps,
            **kwargs
        )
        
    def evaluate(self, steps, **kwargs):
        assert self.model
        assert self._test_gen
        self._log.debug("Evaluating model in test data.")
        
        return self.model.evaluate_generator(
            self._test_gen,
            steps=steps,
            **kwargs
        )

# Model definitions

In [ ]:
def create_model_top_down(input_features, drop_rate=0.5, l2_reg=0.01, 
                          units_factor=128, hidden_layers=2, mode=None):
    mode = mode if mode else 'both_splitted'
    assert mode in ('both_splitted', 'both_fused', 'relation', 'domain')

    normalized_input_features = BatchNormalization()(input_features)
    x = normalized_input_features
    
    if mode != 'relation':
        dom_n_units = int(np.ceil(np.log10(N_CLS_DOMAIN) * units_factor))

        for _ in range(hidden_layers):
            x = Dense(dom_n_units,
                          activation='selu',
                          bias_initializer='lecun_normal',
                          kernel_initializer='lecun_normal',
                          bias_regularizer=l2(l2_reg),
                          kernel_regularizer=l2(l2_reg),              
                      )(x)
            x = AlphaDropout(drop_rate)(x)

        domain = Dense(N_CLS_DOMAIN, name='domain',
                       activation='softmax',
                       bias_regularizer=l2(l2_reg),
                       kernel_regularizer=l2(l2_reg),
                      )(x)

        x = keras.layers.concatenate([normalized_input_features, domain])

    if mode != 'domain':
        rel_n_units = int(np.ceil(np.log10(N_CLS_RELATION) * units_factor))

        for _ in range(hidden_layers):
            x = Dense(rel_n_units,
                          activation='selu',
                          bias_initializer='lecun_normal',
                          kernel_initializer='lecun_normal',
                          bias_regularizer=l2(l2_reg),
                          kernel_regularizer=l2(l2_reg),              
                      )(x)
            x = AlphaDropout(drop_rate)(x)

        relation = Dense(N_CLS_RELATION, name='relation',
                         activation='softmax',
                         bias_regularizer=l2(l2_reg),
                         kernel_regularizer=l2(l2_reg),
                        )(x)

    if mode == 'both_splitted':
        return [domain, relation]
    elif mode == 'both_fused':
        concat = Concatenate(name='domain_relation')
        return [concat([domain, relation])]
    elif mode == 'domain':
        return [domain]
    else:
        return [relation]


def create_model_bottom_up(input_features, drop_rate=0.5, l2_reg=0.01, 
                           units_factor=128, hidden_layers=2, mode=None):
    mode = mode if mode else 'both_splitted'
    assert mode in ('both_splitted', 'both_fused', 'relation', 'domain')

    normalized_input_features = BatchNormalization()(input_features)
    x = normalized_input_features

    # always compute relation
    rel_n_units = int(np.ceil(np.log10(N_CLS_RELATION) * units_factor))
    for _ in range(hidden_layers):
        x = Dense(rel_n_units,
                      activation='selu',
                      bias_initializer='lecun_normal',
                      kernel_initializer='lecun_normal',
                      bias_regularizer=l2(l2_reg),
                      kernel_regularizer=l2(l2_reg),              
                  )(x)
        x = AlphaDropout(drop_rate)(x)

    # domain is a lineal combination of relation
    relation = Dense(N_CLS_RELATION, name='relation',
                     activation='softmax',
                     bias_regularizer=l2(l2_reg),
                     kernel_regularizer=l2(l2_reg),
                    )(x)    
    
    if mode != 'relation':
        # use domain knowledge, weights are frozen
        domain = Dense(N_CLS_DOMAIN, name='domain',
                       activation='linear',
                       use_bias=False, trainable=False,
                       weights=[domain_to_relation_map()],
                      )(relation)

    if mode == 'both_splitted':
        return [domain, relation]
    elif mode == 'both_fused':
        concat = Concatenate(axis=-1, name='domain_relation')
        return [concat([domain, relation])]
    elif mode == 'domain':
        return [domain]
    else:
        return [relation]


class ModelFactory:
    
    def __init__(self, input_shape, image_shape=None):
        self.input_shape = input_shape
        self.image_shape = image_shape if image_shape else input_shape
        self._layer_name_fmt = "{}_attribute_{}"
        
        self._log = logging.getLogger(self.__class__.__name__)

    def create_features(self, model_inputs, models, **kwargs):
        freeze_attributes = kwargs.get('freeze_attributes', False)
        features_layer_idx = kwargs.get('features_layer_idx', None)
        
        self._log.debug('Creating model from inputs {}'.format(model_inputs))

        # collect model inputs and crop them if needed
        inputs, cropped_inputs = self._get_inputs(model_inputs)

        # update layer name and freeze weights if neccesary
        for model_set in models.values():
            for attr_name, model in model_set.items():
                self._update_model(model, attr_name, 
                                   freeze_weights=freeze_attributes)

        # collect features from all attribute models
        feature_outputs = self._get_features(
            list(zip(model_inputs, cropped_inputs)), models,
            features_layer_idx=features_layer_idx
        )
        
        return inputs, feature_outputs

    def create_model(self, inputs, fused_features, version='top_down', **kwargs):        

        if version == 'top_down':
            model_function = create_model_top_down
        elif version == 'bottom_up':
            model_function = create_model_bottom_up
        else:
            self._log.error('Valid models are {top_down, bottom_up}.')
            assert version == 'top_down' or version == 'top_down'
            
        model = Model(
            inputs=inputs,
            outputs=model_function(fused_features, **kwargs)
        )

        return model

    def _get_inputs(self, model_inputs):        
        # cropping dimensions
        diff_shape = np.array(self.image_shape[:2]) - np.array(self.input_shape[:2])
        crop_size = tuple(map(int, diff_shape / 2.0))
        assert crop_size[0] >= 0 and crop_size[1] >= 0

        # collect model inputs and crop them if needed
        inputs, cropped_inputs = [], []
        for input_name in model_inputs:
            _input = Input(shape=self.image_shape, name=input_name)   
            # crop image if neccesary
            if self.input_shape != self.image_shape:
                cropped_input = Cropping2D(
                    crop_size, name="cropped_%s" % input_name
                )(_input)
            else:
                cropped_input = _input

            inputs.append(_input)
            cropped_inputs.append(cropped_input)
            
        return inputs, cropped_inputs
    
    def _get_features(self, named_inputs, models, features_layer_idx=None):
        
        # includes features from all attributes
        feature_outputs = []
        # get image types that are grouped together (e.g. body, face, context)
        im_types = models.keys()
        for im_type in im_types:           
            fused_attrs = self._get_fused_model(
                models[im_type], 
                features_layer_idx=features_layer_idx, 
                name='features_{}'.format(im_type)
            )
            
            # collect features from a given image type
            for input_name, _input in named_inputs:
                if im_type in input_name:
                    self._log.debug('Computing features for input: {}'.format(input_name))

                    attr_input = [_input for _ in fused_attrs.inputs]
                    attr_output = fused_attrs(attr_input)
                    # when using just one attribute, the output is not a list
                    if not isinstance(attr_output, list):
                        attr_output = [attr_output]

                    feature_outputs.extend(attr_output)

        return feature_outputs

    def fuse_features(self, feature_outputs, mode='symmetric', pooling=None):
        fused_features = None

        self._log.debug('Fusing {} outputs'.format(len(feature_outputs)))
        
        if mode == 'symmetric':
            # concatenate features
            fused_features = keras.layers.concatenate(feature_outputs)
        else:
            self._log.error('Valid fusion modes are {symmetric}.')
            assert mode == 'symmetric'
        
        if pooling == 'max':
            # feature downsampling
            fused_features = GlobalMaxPooling2D()(fused_features)
        elif pooling is None:
            fused_features =  Flatten()(fused_features)
        else:
            self._log.error('Valid pooling modes are {None, max}.')
            assert pooling is None or pooling == 'max'
            
        return fused_features
        
    
    def _get_fused_model(self, image_type_models, 
                         features_layer_idx=None, 
                         name=None):
        # includes inputs and outputs for a given type
        attr_inputs, attr_outputs = [], []
        for attr_name, attr_model in image_type_models.items():            
            # collect attribute inputs
            attr_inputs.append(attr_model.input)
            # collect attribute outputs
            if features_layer_idx is None:
                # last layer
                features_layer_idx = -1

            layer = attr_model.layers[features_layer_idx]

            self._log.debug('Collection feature tensors from {} layer {}'.format(attr_name, layer.name))
            
            output = layer.output
            attr_outputs.append(output)

        self._log.debug('Found {} inputs'.format(len(attr_inputs)))
        self._log.debug('Found {} outputs'.format(len(attr_outputs)))
        
        # model that compute features from a given image type
        fused_attrs = Model(inputs=attr_inputs, outputs=attr_outputs, name=name)
        return fused_attrs
        
    def _update_model(self, model, attr_name, freeze_weights=False):
        for layer in model.layers:
            # freeze inherited attribute weights
            layer.trainable = not freeze_weights
            # rename layers if neccesary
            if attr_name not in layer.name:
                layer.name = self._layer_name_fmt.format(layer.name, attr_name)

# Fake call to main to process inputs arguments

In [ ]:
args = [
    "--project_dir", "/home/shared/Documents/final_proj",
    "--save_stats",
    "--save_model",
    "--epochs", "30",
    "--batch_size", "64",
    "--lr", "0.001",
]

conf = main(args)

# Prepate the data

In [ ]:
fakedir = FakeDirectory(conf.FAKE_DIR, conf.SPLITS_DIR)
fakedir.create_structure()

In [ ]:
batch_size = conf.BATCH_SIZE

output_mode = 'both_fused' # domain-relation outputs fused
#output_mode = 'both_splitted' # multi-loss domain-relation
#output_mode = 'domain' # domain only
#output_mode = 'relation' # relation only

# for AlexNet
preprocessing_function = imagenet_utils.preprocess_input

helper = SocialClassifier(conf.FAKE_DIR)
helper.setup_datagen(
    batch_size=batch_size,
    seed=SHARED_SEED,
    classes=RELATION_LABELS,
    target_size=IMAGE_SHAPE[0:2],
    fix_path_cbk=fakedir.fix_fake_link,
    preprocessing_function=preprocessing_function,
    output_mode=output_mode,
)

# Size of each split, output from the data generator

In [ ]:
n_train, n_validation, n_test = 13729, 709, 5106

# Initialize model

In [ ]:
do_reuse_model = conf.REUSE_MODEL
do_create_model = not do_reuse_model

# type of model
# model_version = 'bottom_up'
model_version='top_down'
# initial epoch
cache_epochs = 0

if do_reuse_model:
    checkpoint_path = os.path.join(egosocial.config.MODELS_CACHE_DIR, 'training',
                                   'finetuned_weights.{epoch:02d}-{val_loss:.2f}.h5')
    # set parameters from disk
    cache_epochs, cache_val_loss = None, None
    model_file = checkpoint_path.format(epoch=cache_epochs, val_loss=cache_val_loss)

    # FIXME: doesn't work because the custom loss is an unknown object
    # workaround: save only the weights
    if os.path.exists(model_file):
        pass
    else:
        do_create_model = True
        # start from scratch
        cache_epochs = 0

In [ ]:
if do_create_model:
    # TODO: split submodels in definition and weights
    # load submodels
    selector = AttributeSelector(egosocial.config.MODEL_KEYS)
    attribute_models = dict(
        body=get_models(
            selector.filter('body'), 
            conf.MODELS_CACHE_DIR, 
            conf.MODELS_CACHE_SUBDIR,
        ), 
        face=get_models(
            selector.filter('face'), 
            conf.MODELS_CACHE_DIR, 
            conf.MODELS_CACHE_SUBDIR,
        ),
    )    
    
    # use layer index because layers need to be rename when the 
    # ensemble of models is created
    pool5_idx = -10
    
    model_factory = ModelFactory(INPUT_SHAPE, IMAGE_SHAPE)
    inputs, feature_outputs = model_factory.create_features(
        MODEL_INPUTS, attribute_models,
        freeze_attributes=True,
        features_layer_idx=pool5_idx,
    )
    fused_features = model_factory.fuse_features(
        feature_outputs, 
        pooling='max'
    )
    model = model_factory.create_model(
        inputs, fused_features, 
        mode=output_mode,
        version=model_version,
    )

# Plot model graph

In [ ]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
learning_rate = conf.LR

if output_mode == 'both_fused':
    # we can't use crossentropy because the output
    # is not strictly softmax, but the concatenation
    # of domain and relation outputs (both softmax)
    loss = 'mse'
else:
    loss = 'categorical_crossentropy'
    
helper.set_model(
    model,
    optimizer=keras.optimizers.Adam(learning_rate, decay=1e-6),
    metrics=['accuracy'], loss=loss,
)

# Configure Keras Callbacks

In [ ]:
callbacks = []

if conf.SAVE_MODEL:
    checkpoint_path = os.path.join(egosocial.config.MODELS_CACHE_DIR, 'training',
                                   'finetuned_weights.{epoch:02d}-{val_loss:.2f}.h5')
    checkpointer = ModelCheckpoint( 
        filepath=checkpoint_path, monitor='val_loss',
        save_best_only=True, period=5,
    )
    callbacks.append(checkpointer)

if conf.SAVE_STATS:
    metrics_path = os.path.join(egosocial.config.MODELS_CACHE_DIR, 'training',
                                'metrics.csv')
    csv_logger = CSVLogger(metrics_path)
    callbacks.append(csv_logger)
        
lr_handler = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001
)

# more plots need more space
if output_mode != 'both_splitted':
    figsize = (20, 5)
else:
    figsize = (20, 13)

plot_metrics = PlotLearning(update_step=1, figsize=figsize)

callbacks.extend([
    lr_handler,
    plot_metrics,
])

# Training

In [ ]:
epochs = conf.EPOCHS

hist = helper.fit(
    steps_per_epoch=np.ceil(1.0 * n_train / batch_size),    
    validation_steps=np.ceil(1.0 * n_validation / batch_size),
    epochs=cache_epochs + epochs,
    initial_epoch=cache_epochs,
    callbacks=callbacks,
    use_multiprocessing=False,
    workers=2,
    max_queue_size=5,
    verbose=1,
)

# Evaluation

In [ ]:
scores = helper.evaluate(
    steps=np.ceil(1.0 * n_test / batch_size)
)

for score, metric_name in zip(scores, helper.model.metrics_names):
    helper._log.info("{} : {:0.4}".format(metric_name, score))